In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/driver/chromedriver.exe
/kaggle/input/book-recommendations/Ratings.csv
/kaggle/input/book-recommendations/Books.csv


In [2]:
INDEX_START=8000
INDEX_END=9000

In [3]:
! pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-1.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.


In [4]:
# install google chrome
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

--2024-08-11 19:00:29--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 173.194.69.190, 173.194.69.93, 173.194.69.91, ...
Connecting to dl.google.com (dl.google.com)|173.194.69.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16864 (16K) [application/octet-stream]
Saving to: 'linux_signing_key.pub'

linux_signing_key.p 100%[===================>]  16.47K  --.-KB/s    in 0s      

2024-08-11 19:00:29 (59.2 MB/s) - 'linux_signing_key.pub' saved [16864/16864]

OK
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1825 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:6 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:7 https://packages.cloud.google.com/apt clou

In [5]:
# install chromedriver
!apt-get install -y qq unzip
! wget -O /tmp/chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/127.0.6533.99/mac-arm64/chromedriver-mac-arm64.zip
! unzip /tmp/chromedriver.zip chromedriver-mac-arm64/chromedriver -d /usr/local/bin/




E: Unable to locate package qq
--2024-08-11 19:01:03--  https://storage.googleapis.com/chrome-for-testing-public/127.0.6533.99/mac-arm64/chromedriver-mac-arm64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.207, 142.250.145.207, 74.125.143.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9549773 (9.1M) [application/zip]
Saving to: '/tmp/chromedriver.zip'

/tmp/chromedriver.z 100%[===================>]   9.11M  19.4MB/s    in 0.5s    

2024-08-11 19:01:03 (19.4 MB/s) - '/tmp/chromedriver.zip' saved [9549773/9549773]

Archive:  /tmp/chromedriver.zip
  inflating: /usr/local/bin/chromedriver-mac-arm64/chromedriver  


In [6]:
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [7]:
! google-chrome --version
! chromedriver -version

Google Chrome 127.0.6533.99 
/bin/bash: chromedriver: command not found


In [8]:
books = pd.read_csv('/kaggle/input/book-recommendations/Books.csv')

/tmp/ipykernel_19/2107923939.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/kaggle/input/book-recommendations/Books.csv')


In [9]:
books=books['Book-Title'].tolist()

In [10]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080") 

In [11]:
def get_book_data(index,driver):
    # Using the search bar on the page
    lst=[]
    element = driver.find_element(By.ID, 'mastsearch_query')
    element.clear()
    element.send_keys(books[index])
    element.send_keys(Keys.RETURN)

    # On the search results page now find the url of the book
    url = driver.find_element(By.TAG_NAME, 'body').find_element(By.ID,'lt2_maincontent').find_element(By.ID, 'lt2_content').find_element(By.ID,'lt2_content_interior').find_element(By.ID,'newsearchdiv')

    try:
        ajax_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'ajaxcontent'))  
        )
#         print("AJAX content loaded")
        link = ajax_element.find_element(By.TAG_NAME,'table').find_element(By.TAG_NAME,'tbody').find_element(By.TAG_NAME,'tr').find_element(By.CLASS_NAME,'i').find_element(By.TAG_NAME,'a').get_attribute('href')
        lst.append(books[index])
        lst.append(link)
    except Exception as e:
        print(f"An error occurred: {e}")
        pass
    return lst

In [12]:
import threading 
from tqdm.notebook import tqdm
def thread_function(iterable,thread_id):
    progress_bar = tqdm(iterable,desc=f"Thread {thread_id}")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://www.librarything.com/")
    columns=["Title","Link"]
    df = pd.DataFrame(columns=columns)
    for i in progress_bar:
        print("INDEX = ",i)
        try:
            lst = get_book_data(i,driver)
            df.loc[len(df)] = lst
            progress_bar.set_postfix({"Book":i})
        except:
            pass
    df.to_csv(f"Test_{iterable.start}_{iterable.stop}.csv",index=False)

        
iterables=[
    range(INDEX_START,INDEX_START + (INDEX_END-INDEX_START)//3),
    range(INDEX_START + (INDEX_END-INDEX_START)//3,INDEX_START + (2*(INDEX_END-INDEX_START))//3),
    range(INDEX_START + (2*(INDEX_END-INDEX_START))//3,INDEX_END)
]
                        
threads = []
for i, iterable in enumerate(iterables):
    thread = threading.Thread(target=thread_function, args=(iterable, i))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

Thread 1:   0%|          | 0/333 [00:00<?, ?it/s]

Thread 0:   0%|          | 0/333 [00:00<?, ?it/s]

Thread 2:   0%|          | 0/334 [00:00<?, ?it/s]

INDEX =  8333
INDEX =  8000
INDEX =  8666
INDEX =  8001
INDEX =  8667
INDEX =  8334
INDEX =  8335
INDEX =  8668
INDEX =  8002
INDEX =  8336
INDEX =  8669
INDEX =  8003
INDEX =  8337
INDEX =  8670
INDEX =  8004
INDEX =  8338
INDEX =  8671
INDEX =  8005
An error occurred: Message: no such element: Unable to locate element: {"method":"tag name","selector":"table"}
  (Session info: chrome-headless-shell=127.0.6533.99); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x58d0075866aa <unknown>
#1 0x58d007257600 <unknown>
#2 0x58d0072a6bcb <unknown>
#3 0x58d0072a6eb1 <unknown>
#4 0x58d00729b6a6 <unknown>
#5 0x58d0072c98cd <unknown>
#6 0x58d00729b599 <unknown>
#7 0x58d0072c9a6e <unknown>
#8 0x58d0072e804a <unknown>
#9 0x58d0072c9643 <unknown>
#10 0x58d007299d31 <unknown>
#11 0x58d00729a79e <unknown>
#12 0x58d00754e23b <unknown>
#13 0x58d0075521d2 <unknown>
#14 0x58d00753b5f5 <unknown>